In [304]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [305]:
rename_list = ['izbmun', 'papernum', 'paperdosr', 'paperinside', 'paperoutside', 'papergas', 'paperperen', 'paperstac', 'paperinv', 'paperval', 'paperlost', 'papernc', 'babur', 'grud', 'zhir', 'put', 'sobchak', 'surai', 'titov', 'yavl']
def get_table(parser, tik_soup, region_name, tik_name):
    subsite_url = tik_soup.find('a', text='сайт избирательной комиссии субъекта Российской Федерации')['href']
    tik_page = requests.get(subsite_url)
    tik_soup = BeautifulSoup(tik_page.content, parser)
    table_html = tik_soup.select('td div table').pop()
    table = pd.read_html(str(table_html), header=0).pop()
    table.dropna(inplace=True)
    transposed = table.transpose(copy=False)
    for i in range(13,21):
        transposed[i] = transposed[i].apply(lambda x: int(str(x).split()[0]))
    transposed.columns = rename_list
    transposed.index.name = 'uik'
    transposed.index = transposed.index.map(lambda x: int(x[5:]))
    transposed['region'] = region_name
    transposed['tik'] = tik_name
    return transposed.copy()

In [306]:
%%time
parser = 'html.parser'
# parser = 'lxml'
page = requests.get('http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1&tvd=100100084849066&vrn=100100084849062&region=0&global=1&sub_region=0&prver=0&pronetvd=null&vibid=100100084849066&type=227')
soup = BeautifulSoup(page.content, parser)
tables = []
# for region in [bsobj for bsobj in soup.select('nobr a') if bsobj.get_text() == 'Территория за пределами РФ' or bsobj.get_text() == 'Республика Адыгея (Адыгея)']:
for region in soup.select('nobr a'):
    region_url = region['href']
    region_name = region.get_text()
    region_page = requests.get(region_url)
    region_soup = BeautifulSoup(region_page.content, parser)
    subsite_tag = region_soup.find('a', text='сайт избирательной комиссии субъекта Российской Федерации')
    if (subsite_tag != None):
        print('%s (без ТИКов)' % region_name)
        tables.append(get_table(parser, region_soup, region_name, region_name))
    else:
        print(region_name)
#         for tik in [bsobj for bsobj in region_soup.select('nobr a') if bsobj.get_text() == 'Адыгейская' or bsobj.get_text() == 'Гиагинская']:
        for region in region_soup.select('nobr a'):
            tik_url = tik['href']
            tik_name = tik.get_text()
            tik_page = requests.get(tik_url)
            tik_soup = BeautifulSoup(tik_page.content, parser)
            tables.append(get_table(parser, tik_soup, region_name, tik_name))

tables = pd.concat(tables)
tables.to_csv('D:/data.csv', sep=";", encoding='utf-8')

Республика Адыгея (Адыгея)
Республика Алтай
Республика Башкортостан
Республика Бурятия
Республика Дагестан
Республика Ингушетия
Кабардино-Балкарская Республика
Республика Калмыкия
Карачаево-Черкесская Республика
Республика Карелия
Республика Коми
Республика Крым
Республика Марий Эл
Республика Мордовия
Республика Саха (Якутия)
Республика Северная Осетия - Алания
Республика Татарстан (Татарстан)
Республика Тыва
Удмуртская Республика
Республика Хакасия
Чеченская Республика
Чувашская Республика - Чувашия
Алтайский край
Забайкальский край
Камчатский край
Краснодарский край
Красноярский край
Пермский край
Приморский край
Ставропольский край
Хабаровский край
Амурская область
Архангельская область
Астраханская область
Белгородская область
Брянская область
Владимирская область
Волгоградская область
Вологодская область
Воронежская область
Ивановская область
Иркутская область
Калининградская область
Калужская область
Кемеровская область
Кировская область
Костромская область
Курганская область
Кур